In [3]:
from load_config import read_grinding_config
import numpy as np

## function definition

In [4]:
def selection_function_basic(
    d_i_star: float, 
    a0: float, 
    a1: float, 
    d_crit: float, 
    a2: float
) -> float:
    """
    计算基础选择函数 S_i (Equation 6)
    
    Args:
        d_i_star: 平均粒径 (di* = sqrt(di * di+1))
        a0: 缩放系数 alpha0
        a1: 指数系数 alpha1
        d_crit: 临界粒径 dcrit
        a2: 指数系数 alpha2
    
    Returns:
        选择函数值 S_i
    """
    numerator = a0 * (d_i_star ** a1)
    denominator = 1 + (d_i_star / d_crit) ** a2
    return numerator / denominator


def calculate_d_i_star(d_i: float, d_i_plus_1: float) -> float:
    """
    计算平均粒径 di* = sqrt(di * di+1)
    """
    return np.sqrt(d_i * d_i_plus_1)

def selection_function_expanded(
    d_i_star: float,
    a01: float,
    a11: float,
    a02: float,
    a12: float,
    d_crit: float,
    a2: float
) -> float:
    """
    计算扩展选择函数 S_i (Equation 6e)
    
    Args:
        d_i_star: 平均粒径
        a01: 扩展参数 alpha01
        a11: 扩展参数 alpha11
        a02: 扩展参数 alpha02
        a12: 扩展参数 alpha12
        d_crit: 临界粒径
        a2: 指数系数 alpha2
    
    Returns:
        扩展选择函数值 S_i
    """
    term1_numerator = a01 * (d_i_star ** a11)
    term1_denominator = 1 + (d_i_star / d_crit) ** a2
    term1 = term1_numerator / term1_denominator
    
    term2 = a02 * (d_i_star ** a12)
    
    scaling_factor = 1 / (1 + a02 / a01) if a01 != 0 else 0
    
    return scaling_factor * (term1 + term2)

def breakage_function_basic(
    d_i: float,
    d_j_plus_1: float,
    b0: float,
    b1: float,
    b2: float
) -> float:
    """
    计算基础破碎函数 B_ij (Equation 7)
    
    Args:
        d_i: 原始粒径 di
        d_j_plus_1: 下一粒径 dj+1
        b0: 权重系数 beta0
        b1: 指数系数 beta1
        b2: 指数系数 beta2
    
    Returns:
        破碎函数值 B_ij
    """
    ratio = d_i / d_j_plus_1
    return b0 * (ratio ** b1) + (1 - b0) * (ratio ** b2)

def calculate_beta0j(
    d_j_plus_1: float,  # 单位：μm
    beta00: float,      # 对应论文中的β₀₀
    beta01: float,      # 对应论文中的β₀₁
    max_limit: float = 1.0
) -> float:
    """
    计算动态破碎权重系数 β₀ⱼ (Equation below 7 in original paper)
    
    Args:
        d_j_plus_1: 下一粒径 d_{j+1} (单位：微米μm)
        beta00: 基础系数 (无量纲)
        beta01: 衰减指数 (无量纲)
        max_limit: β₀ⱼ的最大值限制 (默认1.0)
    
    Returns:
        动态权重系数 β₀ⱼ ∈ [0, max_limit]
    
    Notes:
        1. 公式原型：β₀ⱼ = β₀₀·(d_{j+1}/1000)^{-β₀₁}
        2. 隐含单位转换：分母1000将μm转换为mm
        3. 根据论文要求，β₀ⱼ不得超过1.0
    """
    beta0j = beta00 * (d_j_plus_1 / 1000) ** (-beta01)
    return min(beta0j, max_limit)

def breakage_function_expanded(
    d_i: float,          # 原始粒径 d_i (μm)
    d_j_plus_1: float,   # 下一粒径 d_{j+1} (μm)
    beta00: float,       # 对应论文参数
    beta01: float,       # 对应论文参数
    beta1: float,       # 对应论文中的β₁
    beta2: float        # 对应论文中的β₂
) -> float:
    """
    计算扩展破碎函数 B_{ij} (Equation 7 with dynamic β₀ⱼ)
    
    Args:
        d_i: 原始粒径 (μm)
        d_j_plus_1: 参考粒径 (μm)
        beta00: 基础权重系数
        beta01: 粒径衰减指数
        beta1: 粗粒组分指数
        beta2: 细粒组分指数
    
    Returns:
        破碎概率 B_{ij} ∈ [0,1]
    """
    beta0j = calculate_beta0j(d_j_plus_1, beta00, beta01)
    # print(beta0j)
    ratio = d_i / d_j_plus_1
    return beta0j * (ratio ** beta1) + (1 - beta0j) * (ratio ** beta2)

In [5]:
def calculate_slurry_density(
    percent_solids: float,  # 浓度百分比（0-100%）
    ore_density: float,     # 浓度密度（ton/m³）
    water_density: float = 1.00  # 水的密度（ton/m³），默认1.0
) -> float:
    """
    计算矿浆密度（Slurry Density）
    
    公式：
    ρ_slurry = 1 / [ (w_solids/ρ_ore) + (w_water/ρ_water) ]
    其中 w_solids = %_solids / 100, w_water = 1 - w_solids

    Args:
        percent_solids: 浓度质量百分比（%）
        ore_density: 浓度密度（ton/m³）
        water_density: 液体密度（ton/m³），默认为1.0（水）

    Returns:
        矿浆密度（ton/m³）

    Raises:
        ValueError: 如果输入百分比或密度无效
    """
    # 输入验证
    if not 0 <= percent_solids <= 100:
        raise ValueError("浓度百分比必须在0-100之间")
    if ore_density <= 0 or water_density <= 0:
        raise ValueError("密度必须为正数")
    
    # 计算质量分数
    w_solids = percent_solids / 100
    w_water = 1 - w_solids
    
    # 计算倒数密度
    inverse_density = (w_solids / ore_density) + (w_water / water_density)
    
    # 返回矿浆密度
    return 1 / inverse_density

In [22]:
def calculate_mill_operation(
    D_ft: float,            # 磨机有效直径 (ft)
    L_ft: float,            # 磨机有效长度 (ft)
    Nc_pct: float,          # 转速 (%临界转速)
    J: float,               # 总填充率 (小数)
    Jb: float,              # 钢球填充率 (小数)
    Jp: float,              # 间隙矿浆填充率 (小数)
    fs: float,              # 矿浆浓度百分比 (小数)
    rb: float,              # 钢球密度 (ton/m3)
    rm: float,              # 矿石密度 (ton/m3)
    alpha_deg: float,       # 提升角 (度)
    fv: float = 0.4,        # 钢球间隙率 (默认0.4)
    eta: float = 0.9        # 传动效率 (默认0.9)
) -> dict:
    """
    综合计算磨机装料参数和功率特性
    
    Returns:
        {
            'charge_parameters': {
                'mill_volume': 磨机总体积(m3),
                'apparent_volume': 表观装料体积(m3),
                'charge_weight': {
                    'total': 总重量(ton),
                    'balls': 钢球重量(ton),
                    'interstitial_slurry': 间隙矿浆重量(ton),
                    'overfilling_slurry': 溢流矿浆重量(ton)
                },
                'apparent_density': 表观密度(ton/m3)
            },
            'power_parameters': {
                'net_power': 净功率(kW),
                'gross_power': 总功率(kW),
                'power_breakdown': {
                    'balls': 钢球贡献功率(kW),
                    'interstitial_slurry': 间隙矿浆贡献功率(kW),
                    'overfilling_slurry': 溢流矿浆贡献功率(kW)
                },
                'intermediate_values': {
                    'critical_speed': 临界转速(rpm),
                    'actual_speed': 实际转速(rpm),
                    'slurry_density': 矿浆密度(ton/m3)
                }
            }
        }
    """
    # ========== 1. 单位转换 ==========
    D = D_ft * 0.305  # ft → m
    L = L_ft * 0.305
    
    # ========== 2. 磨机体积计算 ==========
    V_mill = np.pi * (D/2)**2 * L
    
    # ========== 3. 矿浆密度计算 ==========
    rp = 1 / (fs/rm + (1-fs)/1.0)  # 水密度=1.0
    
    # ========== 4. 装料参数计算 ==========
    # 各组分的质量(ton)
    M_balls = (1-fv) * rb * Jb * V_mill
    M_inter = rp * Jp * fv * Jb * V_mill
    M_over = rp * (J-Jb) * V_mill
    M_total = M_balls + M_inter + M_over
    
    # 表观体积和密度
    V_apparent = J * V_mill
    rap = M_total / V_apparent
    
    # ========== 5. 功率计算 ==========
    # 临界转速 (rpm)
    Ncrit = 76.6 / np.sqrt(D_ft)
    
    # 实际转速 (小数 fraction of critical)
    Nc = Nc_pct / 100
    
    # 净功率 (kW)
    Pnet = 0.238 * (D_ft**3.5) * (L_ft/D_ft) * Nc * rap * (J - 1.065*J**2) * np.sin(np.radians(alpha_deg))
    
    # 总功率 (kW)
    Pgross = Pnet / eta
    
    # 功率组分分解
    Pb = ((1-fv)*rb*Jb) / (rap*J) * (eta*Pgross)
    Ps = (rp*Jp*fv*Jb) / (rap*J) * (eta*Pgross)
    Po = (rp*(J-Jb)) / (rap*J) * (eta*Pgross)
    
    return {
        'charge_parameters': {
            'mill_volume': V_mill,
            'apparent_volume': V_apparent,
            'charge_weight': {
                'total': M_total,
                'balls': M_balls,
                'interstitial_slurry': M_inter,
                'overfilling_slurry': M_over
            },
            'apparent_density': rap
        },
        'power_parameters': {
            'net_power': Pnet,
            'gross_power': Pgross,
            'power_breakdown': {
                'balls': Pb,
                'interstitial_slurry': Ps,
                'overfilling_slurry': Po
            },
            'intermediate_values': {
                'critical_speed': Ncrit,
                'actual_speed': Ncrit * Nc,
                'slurry_density': rp
            }
        }
    }

In [24]:
import math
import numpy as np
from typing import List, Dict

class CIMMCycloneModel:
    def __init__(self, a_params: Dict[str, float], lambda_val: float):
        """初始化模型参数
        Args:
            a_params: 模型系数 {'a1':, 'a2':, 'a3':, 'a4':}
            lambda_val: 短路比例系数 λ
        """
        self.a1 = a_params.get('a1', 1.0)
        self.a2 = a_params.get('a2', 1.0)
        self.a3 = a_params.get('a3', 1.0)
        self.a4 = a_params.get('a4', 1.0)
        self.lambda_val = lambda_val

    def correlation1_pressure(self, Q: float, phi: float, 
                            DC: float, h: float, 
                            DI: float, DO: float, DU: float) -> float:
        """关联式1：计算旋流器入口压力
        Args:
            Q: 体积给矿量 (m³/hr)
            phi: 给矿体积浓度分数
            DC: 旋流器直径 (inches)
            h: 自由高度 (inches)
            DI: 入口直径 (inches)
            DO: 溢流管直径 (inches)
            DU: 底流口直径 (inches)
        Returns:
            入口压力 (ft of slurry)
        """
        numerator = Q**1.46 * math.exp(-7.63*phi + 10.79*phi**2)
        denominator = DC**0.20 * h**0.15 * DI**0.51 * DO**1.65 * DU**0.53
        return self.a1 * numerator / denominator

    def correlation2_cut_size(self, DC: float, DI: float, DO: float, DU: float,
                            h: float, Q: float, phi: float, rho_s: float) -> float:
        """关联式2：计算校正切割尺寸d50c
        Args:
            rho_s: 浓度密度 (ton/m³)
        Returns:
            校正切割尺寸 (μm)
        """
        numerator = DC**0.44 * DI**0.58 * DO**1.91 * math.exp(11.12*phi)
        denominator = DU**0.80 * h**0.37 * Q**0.44 * (rho_s - 1)**0.5
        return self.a2 * numerator / denominator

    def correlation3_flow_split(self, h: float, DU: float, DO: float, 
                              phi: float, H: float, DC: float) -> float:
        """关联式3：计算流量分配比S (底流/溢流)"""
        numerator = h**0.19 * (DU/DO)**2.64 * math.exp(-4.33*phi + 8.77*phi**2)
        denominator = H**0.54 * DC**0.38
        return self.a3 * numerator / denominator

    def correlation4_corrected_efficiency(self, di: float, d50c: float, m: float) -> float:
        """关联式4：计算校正分级效率Eic"""
        return 1 - math.exp(-0.693 * (di/d50c)**m)

    def calculate_plitts_m(self, S: float, DC: float, h: float, Q: float) -> float:
        """计算Plitt参数m"""
        return math.exp(self.a4 - 1.58*S/(S+1)) * ((DC**2 * h)/Q)**0.15

    def correlation5_bypass(self, S: float, phi: float, 
                          feed_size_dist: List[float], 
                          eic_values: List[float]) -> Dict[str, float]:
        """关联式5：计算短路效应
        Args:
            feed_size_dist: 给矿粒度分布 [f1, f2,...]
            eic_values: 对应粒度的Eic值
        Returns:
            {'Bpf': , 'Bpw': , 'Rsc': }
        """
        # 计算Rsc
        Rsc = sum(f*eic for f, eic in zip(feed_size_dist, eic_values))
        
        # 计算水短路Bpw
        numerator = S/(S+1) - phi*Rsc
        denominator = 1 - phi*(1 - self.lambda_val*(1 - Rsc))
        Bpw = numerator / denominator
        
        # 计算矿浆短路Bpf
        Bpf = self.lambda_val * Bpw
        
        return {'Bpf': Bpf, 'Bpw': Bpw, 'Rsc': Rsc}

    def calculate_partition_curve(self, size_dist: List[float], 
                                d50c: float, m: float, Bpf: float) -> List[float]:
        """计算实际分配曲线Ei
        Args:
            size_dist: 粒度列表 [d1, d2,...] (μm)
        Returns:
            各粒度对应的分配率 [Ei1, Ei2,...]
        """
        return [Bpf + (1 - Bpf) * self.correlation4_corrected_efficiency(d, d50c, m)
                for d in size_dist]

### 溢流器参数

In [48]:
for i in range(20):
    d_i = float(params['mesh_'+str(i+1)]['opening'])
    d_ip1 = float(params['mesh_' + str(i+2)]['opening'])
    d_i_star = np.sqrt(d_i*d_ip1)

In [59]:
params['mesh_1']

{'mesh': '1.05',
 'opening': '25400',
 'mid_size': '30206',
 'ton_hr': 0.0,
 'percent_retained': 0.0,
 'percent_passing': 100.0}

In [77]:
# 初始化模型
model = CIMMCycloneModel(params, params['lambda'])

# 输入参数
Q = 1565.75/10      # m³/hr
phi = 0.37     # 体积浓度分数
DC, h = 20, 75 # inches
DI, DO, DU = 3.5, 7.5, 3.67  # inches
rho_s = 2.8    # ton/m³

# 1. 计算入口压力
H = model.correlation1_pressure(Q, phi, DC, h, DI, DO, DU)

# 2. 计算切割尺寸
d50c = model.correlation2_cut_size(DC, DI, DO, DU, h, Q, phi, rho_s)

# 3. 计算流量分配比
S = model.correlation3_flow_split(h, DU, DO, phi, H, DC)

# 4. 计算Plitt's m参数
m = model.calculate_plitts_m(S, DC, h, Q)

# 5. 定义粒度分布
size_dist = []
feed_dist = [] # 各粒度质量分数

passings = [100.00, 100.00, 98.88, 96.32, 93.81, 91.53, 89.16, 86.44, 83.28, 79.16, 
            74.30, 68.10, 60.53, 51.77, 42.92, 35.18, 29.08, 24.57, 21.24, 18.84, 0.00]
for i in range(21):
    d_i_star = float(params['mesh_'+str(i+1)]['mid_size'])

    if i == 0:
        feed_dist.append(0)
    else:
        passing_i = passings[i-1]#float(params['mesh_'+str(i)]['percent_passing'])
        passing_i1 = passings[i]#float(params['mesh_'+str(i+1)]['percent_passing'])
        passing = (passing_i - passing_i1)
        feed_dist.append(passing/100)

    size_dist.append(d_i_star)
    

# 6. 计算各粒度Eic
eic_values = [model.correlation4_corrected_efficiency(d, d50c, m) for d in size_dist]

# 7. 计算短路参数
bypass = model.correlation5_bypass(S, phi, feed_dist, eic_values)

# 8. 计算实际分配曲线
partition = model.calculate_partition_curve(size_dist, d50c, m, bypass['Bpf'])

# 打印结果
print(f"1. 入口压力 H = {H:.5f} ft of slurry")
print(f"2. 校正切割尺寸 d50c = {d50c:.5f} μm")
print(f"3. 流量分配比 S = {S:.5f}") 
print(f"4. Plitt参数 m = {m:.5f}")
# print(eic_values)
print(f"5. 短路参数 - Bpf = {bypass['Bpf']:.5f}, Bpw = {bypass['Bpw']:.5f}, Rsc = {bypass['Rsc']:.5f}")
print("\n实际分配曲线:")
for d, ei in zip(size_dist, partition):
    print(f"  {d} μm: {ei:.5f}")


1. 入口压力 H = 11.05523 ft of slurry
2. 校正切割尺寸 d50c = 179.41527 μm
3. 流量分配比 S = 1.10819
4. Plitt参数 m = 1.61732
5. 短路参数 - Bpf = 0.37212, Bpw = 0.39171, Rsc = 0.60779

实际分配曲线:
  30206.0 μm: 1.00000
  21997.0 μm: 1.00000
  15554.0 μm: 1.00000
  10984.0 μm: 1.00000
  7978.0 μm: 1.00000
  5641.0 μm: 1.00000
  3989.0 μm: 1.00000
  2812.0 μm: 1.00000
  2003.0 μm: 1.00000
  1416.0 μm: 1.00000
  1001.0 μm: 0.99999
  714.0 μm: 0.99903
  505.0 μm: 0.98440
  357.0 μm: 0.92377
  252.0 μm: 0.81098
  178.0 μm: 0.68323
  126.0 μm: 0.57544
  89.0 μm: 0.49763
  63.0 μm: 0.44730
  45.0 μm: 0.41691
  19.0 μm: 0.38354


### 磨机功率和过程参数

In [23]:

params = {
    'D_ft': 18.5,
    'L_ft': 22.0,
    'Nc_pct': 72.0,
    'J': 0.38,
    'Jb': 0.38,
    'Jp': 1.0,
    'fs': 0.72,
    'rb': 7.75,
    'rm': 2.8,
    'alpha_deg': 35.0
}

results = calculate_mill_operation(**params)

# 装料参数
charge = results['charge_parameters']
print("===== 装料参数 =====")
print(f"磨机总体积: {charge['mill_volume']:.2f} m3")
print(f"表观装料体积: {charge['apparent_volume']:.2f} m3")
print(f"钢球重量: {charge['charge_weight']['balls']:.2f} ton")
print(f"间隙矿浆重量: {charge['charge_weight']['interstitial_slurry']:.2f} ton")
print(f"溢流矿浆重量: {charge['charge_weight']['overfilling_slurry']:.2f} ton")
print(f"总重量: {charge['charge_weight']['total']:.2f} ton")
print(f"表观密度: {charge['apparent_density']:.3f} ton/m3")

# 功率参数
power = results['power_parameters']
print("\n===== 功率参数 =====")
print(f"临界转速: {power['intermediate_values']['critical_speed']:.2f} rpm")
print(f"实际转速: {power['intermediate_values']['actual_speed']:.2f} rpm")
print(f"矿浆密度: {power['intermediate_values']['slurry_density']:.3f} ton/m3")
print(f"\n净功率: {power['net_power']:.1f} kW")
print(f"总功率: {power['gross_power']:.1f} kW")
print("\n功率分布:")
print(f"- 钢球贡献: {power['power_breakdown']['balls']:.1f} kW ({power['power_breakdown']['balls']/power['net_power']*100:.1f}%)")
print(f"- 间隙矿浆贡献: {power['power_breakdown']['interstitial_slurry']:.1f} kW ({power['power_breakdown']['interstitial_slurry']/power['net_power']*100:.1f}%)")
print(f"- 溢流矿浆贡献: {power['power_breakdown']['overfilling_slurry']:.1f} kW ({power['power_breakdown']['overfilling_slurry']/power['net_power']*100:.1f}%)")

===== 装料参数 =====
磨机总体积: 167.79 m3
表观装料体积: 63.76 m3
钢球重量: 296.48 ton
间隙矿浆重量: 47.48 ton
溢流矿浆重量: 0.00 ton
总重量: 343.96 ton
表观密度: 5.395 ton/m3

===== 功率参数 =====
临界转速: 17.81 rpm
实际转速: 12.82 rpm
矿浆密度: 1.862 ton/m3

净功率: 3884.5 kW
总功率: 4316.1 kW

功率分布:
- 钢球贡献: 3348.3 kW (86.2%)
- 间隙矿浆贡献: 536.2 kW (13.8%)
- 溢流矿浆贡献: 0.0 kW (0.0%)


### 选择函数和破碎函数

In [78]:
params = read_grinding_config('grinding_config.ini', flatten=True)


In [79]:
dcrit = params['dcrit']
alpha01 = alpha0 = params['alpha0']
alpha11 = alpha1 = params['alpha1']
alpha2 = params['alpha2']

alpha02 = params['alpha02']
alpha12 = params['alpha12']

beta00 = params['beta0']
beta01 = params['beta01']
beta1 = params['beta1']
beta2 = params['beta2']

ore_density_mill = params['ore (ton/m3)']
solids_mill = params['mill discharge (%)']
ball_dnesity_mill = params['balls (ton/m3)']

In [80]:
ball_dnesity_mill

7.75

In [81]:
calculate_slurry_density(
    percent_solids=solids_mill,
    ore_density=ore_density_mill,
    water_density=1.00
)


1.8617021276595744

In [121]:
n = 19

siE_array = np.zeros(n)
bij_cummula = np.zeros((n+1, n+1))
np.fill_diagonal(bij_cummula, 1)
bij_partial = np.zeros((n+1, n+1))

for i in range(len(siE_array)):
    d_i = float(params['mesh_'+str(i+1)]['opening'])
    d_ip1 = float(params['mesh_' + str(i+2)]['opening'])
    d_i_star = np.sqrt(d_i*d_ip1)
    
    siE = selection_function_expanded(d_i_star, alpha01, alpha11, alpha02, alpha12, dcrit, alpha2)
    siE_array[i] = siE
    
    for j in range(len(siE_array)):
        if not j > i:
            d_j_plus_1 = float(params['mesh_' + str(j+2)]['opening'])
            bij = breakage_function_expanded(d_ip1, d_j_plus_1, beta00, beta01, beta1, beta2)
            bij_cummula[i+1][j] = bij
            bij_partial[i][j] = bij_cummula[i][j] - bij_cummula[i+1][j]

bij_partial[-1, :] = bij_cummula[-1, :]
bij_partial[-1, -1] = 0

siE_array = np.append(siE_array, 0)
print("siE array:", siE_array)
print("bij array:\n", bij_cummula)
print("bij partial array:\n", bij_partial)


siE array: [0.27970586 0.49950547 0.83236684 1.19137926 1.48778328 1.55715388
 1.42840647 1.22149147 1.00391042 0.81149568 0.65477173 0.52389851
 0.41862983 0.33407122 0.26673378 0.21294834 0.17000142 0.13571357
 0.10880787 0.        ]
bij array:
 [[1.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.        ]
 [1.         1.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.        ]
 [0.33874509 1.         1.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.        ]
 [0.21754595 0.43647669 1.         1.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.  

### J&T

In [122]:
import numpy as np
from scipy.linalg import toeplitz

def compute_grinding_matrices_vectorized(S, B, t, N=None, mode='batch', eps=1e-10):
    """
    向量化计算磨矿矩阵T和J
    
    参数:
        S (np.array): 选择函数向量 (n,)
        B (np.array): 破碎函数矩阵 (n,n)
        t (float): 平均停留时间
        N (int): 连续磨矿混合器数量
        mode (str): 'batch'或'continuous'
        eps (float): 数值稳定的小量
        
    返回:
        tuple: (T_matrix, J_matrix)
    """
    n = len(S)
    S = S.reshape(-1, 1)  # 转换为列向量 (n,1)
    
    # ===== 1. 向量化计算矩阵T =====
    # 生成i和j的索引矩阵 (n,n)
    i, j = np.indices((n, n))
    
    # 初始化T矩阵
    T = np.zeros((n, n))
    
    # 对角线元素=1
    np.fill_diagonal(T, 1)
    
    # 计算i>j的部分
    mask = i > j
    i_idx, j_idx = np.where(mask)
    
    # 预计算所有k值 (利用广播机制)
    k = np.arange(n).reshape(1, -1)  # (1,n)
    valid_k = (k >= j_idx.reshape(-1, 1)) & (k < i_idx.reshape(-1, 1))  # (n_cond, n)
    
    # 向量化计算求和项
    S_k = S[k]  # (n_cond, n, 1)
    B_ik = B[i_idx[:, None], k]  # (n_cond, n)
    T_kj = T[k, j_idx[:, None]]  # (n_cond, n)
    
    numerator = B_ik * S_k.squeeze() * T_kj  # (n_cond, n)
    denominator = S[i_idx] - S[j_idx] + eps  # 添加小量防止除零 (n_cond, 1)
    
    sum_terms = np.where(valid_k, numerator / denominator, 0)  # (n_cond, n)
    T[i_idx, j_idx] = np.sum(sum_terms, axis=1)  # 沿k轴求和
    
    # ===== 2. 向量化计算矩阵J =====
    J = np.diag(
        np.exp(-S.flatten() * t) if mode == 'batch' 
        else (1 + S.flatten() * t / N) ** (-N)
    )
    
    return T, J

In [126]:
t = 1.966338569
N = 3.874856083
T_vec, J_vec = compute_grinding_matrices_vectorized(siE_array, bij_partial, t=t, N=N, mode='continuous')

print("向量化结果:")
print(f"T =\n{T_vec[:5, :5]}")
print(f"J =\n{J_vec}")

向量化结果:
T =
[[1.         0.         0.         0.         0.        ]
 [0.84147952 1.         0.         0.         0.        ]
 [0.06133979 0.84564627 1.         0.         0.        ]
 [0.01573477 0.14732137 1.43467378 1.         0.        ]
 [0.00505643 0.03050713 0.20703772 2.46946569 1.        ]]
J =
[[0.59791688 0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.        ]
 [0.         0.41668679 0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.        ]
 [0.         0.         0.25531137 0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.        ]
 [0.         0.         0.         0.16005001 0.         0.
  0.         0.    